# Fit the Model

In [104]:
%%sh
drum fit --code-dir ./model \
--input data/10K_Lending_Club_Loans_utf-8.csv \
--output ./model \
--target-type binary \
--target is_bad \
--docker env

Files were overwritten: {'/opt/fit_output_dir/create_pipeline.py', '/opt/fit_output_dir/artifact.pkl', '/opt/fit_output_dir/custom.py', '/opt/fit_output_dir/feature_detail.yaml', '/opt/fit_output_dir/schema.json', '/opt/fit_output_dir/create_data.py', '/opt/fit_output_dir/README.md', '/opt/fit_output_dir/model-config.yaml', '/opt/fit_output_dir/custom_model.py'}
Success 🎉
Host DRUM version: 1.4.4
Container DRUM version: 
the input device is not a TTY



# Testing the model

## Performance Testing

In [18]:
%%sh
drum perf-test \
--code-dir ./model \
--input "data/10K_LC_short.csv" \
--target-type binary \
--positive-class-label 1 \
--negative-class-label 0 \
--docker env \
--verbose

Detected perf-test mode
DRUM performance test
Model:      /Users/timothy.whittaker/Desktop/git/ford-mlops/model
Data:       /Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_LC_short.csv
# Features: 35
Preparing test data...
Running drum using: [drum server --code-dir /Users/timothy.whittaker/Desktop/git/ford-mlops/model --address localhost:64316 --logging-level warning --show-perf --target-type binary --positive-class-label 1 --negative-class-label 0 --docker env]



Running test case with timeout: 180
Running test case: 550 bytes - 1 samples, 100 iterations
Processing |################################| 100/100
Running test case with timeout: 180
Running test case: 0.1MB - 190 samples, 50 iterations
Processing |################################| 50/50
Running test case with timeout: 180
Running test case: 10MB - 19051 samples, 5 iterations
Processing |################################| 5/5
Running test case with timeout: 180
Running test case: 50MB - 95256 samples, 1 iterations
P

## Validation

In [20]:
%%sh
drum validation \
--code-dir ./model \
--input "data/10K_LC_short.csv" \
--target-type binary \
--positive-class-label 1 \
--negative-class-label 0 \
--docker env > validation.log

docker: Error response from daemon: invalid mode: /opt/input.csv.
See 'docker run --help'.
2020-12-15 11:41:11,043 ERROR drum:  Error from docker process: 125
Traceback (most recent call last):
  File "/usr/local/bin/drum", line 6, in <module>
    main()
  File "/usr/local/lib/python3.7/site-packages/datarobot_drum/drum/main.py", line 96, in main
    CMRunner(runtime).run()
  File "/usr/local/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 289, in run
    self._run_fit_and_predictions_pipelines_in_mlpiper()
  File "/usr/local/lib/python3.7/site-packages/datarobot_drum/drum/drum.py", line 559, in _run_fit_and_predictions_pipelines_in_mlpiper
    _pipeline_executor.run_pipeline(cleanup=False)
  File "/usr/local/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 261, in run_pipeline
    self._run_pipeline()
  File "/usr/local/lib/python3.7/site-packages/mlpiper/pipeline/executor.py", line 251, in _run_pipeline
    self._dag.run_connected_pipeline(self._ml_engin

In [22]:
! tail -n 30 validation.log

the input device is not a TTY



Validation checks results
      Test case         Status                       Details                    
Null value imputation   FAILED   Null value imputation check performs check by  
                                 imputing each feature with NaN value. If check 
                                 fails for a feature, test dataset is saved in  
                                 /tmp/drum_validation_checks_. Make sure to     
                                 delete those folders if it takes too much      
                                 space.                                         
                                                                                
                                 Failed feature                          Dataset
                                 filename                                       
                                 Unnamed: 0       /tmp/drum_validation_checks_qo
                                 6vnck_/null_value

## Batch Scoring

In [95]:
%%sh
drum score \
--code-dir ./model \
--target-type binary \
--positive-class-label pos_class_prediction \
--negative-class-label neg_class_prediction \
--input data/10K_LC_short.csv \
--docker env \
--verbose --output data/predictions.csv

Detected score mode
Detected /opt/model/custom.py .. trying to load hooks
Finished loading model, total used 100 iterations
 
 
Component: generic_predictor
Language:  Python
Output:
------------------------------------------------------------
{'target_type': <TargetType.BINARY: 'binary'>, 'positive_class_label': 'pos_class_prediction', 'negative_class_label': 'neg_class_prediction'}
------------------------------------------------------------
Runtime:    0.1 sec
NR outputs: 0
 
docker command: [docker run --rm --interactive --user $(id -u):$(id -g)   -v /Users/timothy.whittaker/Desktop/git/ford-mlops/model:/opt/model -v "/Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_LC_short.csv":/opt/input.csv -v "/Users/timothy.whittaker/Desktop/git/ford-mlops/data/predictions.csv":/opt/output.csv sha256:d796fdc6e00d9b1b32519ccd3e39b93d648769b66bdbdf255aa4f33a3ca24b37 drum score --code-dir /opt/model --target-type binary --positive-class-label pos_class_prediction --negative-class-label n

In [96]:
!head -n 5 data/predictions.csv

neg_class_prediction,pos_class_prediction
0.9897753296269703,0.010224670373029644
0.9624397300705587,0.03756026992944123
0.9730381523965298,0.02696184760347018
0.9618944584147967,0.038105541585203254


## Batch Scoring Unstructured Model

Returns Predictions and SHAP Values

In [97]:
%%sh
drum score \
--code-dir ./model \
--input "data/10K_LC_short.csv" \
--target-type unstructured \
--content-type application/text \
--docker env \
--verbose --output data/predictions.json

Detected score mode
Detected /opt/model/custom.py .. trying to load hooks
Finished loading model, total used 100 iterations
 
 
Component: generic_predictor
Language:  Python
Output:
------------------------------------------------------------
Incoming content type params:  {'mimetype': 'application/text'}
Incoming data type:  <class 'bytes'>
Incoming query params:  {}
------------------------------------------------------------
Runtime:    0.2 sec
NR outputs: 0
 
docker command: [docker run --rm --interactive --user $(id -u):$(id -g)   -v /Users/timothy.whittaker/Desktop/git/ford-mlops/model:/opt/model -v "/Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_LC_short.csv":/opt/input.csv -v "/Users/timothy.whittaker/Desktop/git/ford-mlops/data/predictions.json":/opt/output.csv sha256:d796fdc6e00d9b1b32519ccd3e39b93d648769b66bdbdf255aa4f33a3ca24b37 drum score --code-dir /opt/model --input /opt/input.csv --target-type unstructured --content-type application/text --verbose --output /o

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


In [98]:
!head -n 5 data/predictions.json

{"data": [{"predictions": {"negative_class_label_prob": 0.9897753296269703, "positive_class_label_prob": 0.010224670373029644}, "shap_values": {"POS_CLASS_SHAP_int_rate": -0.943694117200347, "POS_CLASS_SHAP_loan_amnt": -0.07461917262785686, "POS_CLASS_SHAP_funded_amnt": 0.003682842085621606, "POS_CLASS_SHAP_installment": 0.024326240027360016, "POS_CLASS_SHAP_annual_inc": -0.08212664029965298, "POS_CLASS_SHAP_dti": -0.01653197270073701, "POS_CLASS_SHAP_delinq_2yrs": -0.0028207938655536993, "POS_CLASS_SHAP_inq_last_6mths": 0.027145089367756815, "POS_CLASS_SHAP_mths_since_last_delinq": -0.030683564379422688, "POS_CLASS_SHAP_mths_since_last_record": -0.00781557856516893, "POS_CLASS_SHAP_open_acc": 0.04909428369960561, "POS_CLASS_SHAP_pub_rec": 0.0, "POS_CLASS_SHAP_revol_bal": 0.045280461036655534, "POS_CLASS_SHAP_revol_util": -0.19955666934516722, "POS_CLASS_SHAP_total_acc": -0.04077275682180262, "POS_CLASS_SHAP_mths_since_last_major_derog": 0.0, "POS_CLASS_SHAP_policy_code": 0.0, "POS_CLA

12237467, "POS_CLASS_SHAP_loan_amnt": 0.14375167395929134, "POS_CLASS_SHAP_funded_amnt": 0.06698104622095843, "POS_CLASS_SHAP_installment": 0.01886063574985922, "POS_CLASS_SHAP_annual_inc": -0.6652168252420366, "POS_CLASS_SHAP_dti": 0.04680774321786714, "POS_CLASS_SHAP_delinq_2yrs": -0.00657730601607602, "POS_CLASS_SHAP_inq_last_6mths": -0.011777898949033938, "POS_CLASS_SHAP_mths_since_last_delinq": -0.0009383883103690146, "POS_CLASS_SHAP_mths_since_last_record": -0.015223993707371982, "POS_CLASS_SHAP_open_acc": -0.09199893702982202, "POS_CLASS_SHAP_pub_rec": 0.0, "POS_CLASS_SHAP_revol_bal": 0.005012798407174372, "POS_CLASS_SHAP_revol_util": 0.018853619919601317, "POS_CLASS_SHAP_total_acc": -0.011903213568832663, "POS_CLASS_SHAP_mths_since_last_major_derog": 0.0, "POS_CLASS_SHAP_policy_code": 0.0, "POS_CLASS_SHAP_term": 0.5014208619898651, "POS_CLASS_SHAP_grade": 0.017642366530292622, "POS_CLASS_SHAP_sub_grade": 0.034033723368052826, "POS_CLASS_SHAP_emp_title": -0.0887574161699219, "PO

## Serving Model

In [57]:
import subprocess
import requests
import time

In [53]:
run_inference_server = [
    "drum", "server",
    "--code-dir","./model", 
    "--address", "0.0.0.0:6789", 
    "--show-perf",
    "--target-type", "binary",
    "--logging-level", "info",
    "--show-stacktrace",
    "--verbose"
]
inference_server = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

In [54]:
## snoozing before pinging the server to give it time to actually start
time.sleep(5)
print('check status')
requests.request("GET", "http://0.0.0.0:6789/").content

check status


b'{"message":"OK"}\n'

In [55]:
## shutdown
requests.request("POST", "http://0.0.0.0:6789/shutdown/").content

b'Server shutting down...'

In [56]:
inference_server.terminate()
inference_server.stdout.readlines()

[b'Detected REST server mode - this is an advanced option\n',
 b'Detected /Users/timothy.whittaker/Desktop/git/ford-mlops/model/custom.py .. trying to load hooks\n',
 b'\x1b \x1b\n',
 b'\x1b \x1b\n',
 b'\x1b============================================================\x1b\n',
 b'\x1bComponent: prediction_server\x1b\n',
 b'\x1bLanguage:  Python\x1b\n',
 b'\x1bOutput:\x1b\n',
 b'\x1b------------------------------------------------------------\x1b\n',
 b' * Serving Flask app "datarobot_drum.drum.server" (lazy loading)\n',
 b' * Environment: production\n',
 b'   WARNING: This is a development server. Do not use it in a production deployment.\n',
 b'   Use a production WSGI server instead.\n',
 b' * Debug mode: off\n']

## Push Artifacts and Environment to DataRobot

In [80]:
!python push.py --env-dir ./env --model-dir ./model --logging-level INFO --max-wait 600 --update-env True

Namespace(env_dir='./env', logging_level='INFO', max_wait=600, model_dir='./model', update_env=True)
__main__ - INFO - 2020-12-16 08:42:21,553 - Namespace(env_dir='./env', logging_level='INFO', max_wait=600, model_dir='./model', update_env=True)
__main__ - INFO - 2020-12-16 08:42:21,855 - load model config
__main__ - INFO - 2020-12-16 08:42:21,864 - {'name': 'lgbm-classifier', 'type': 'inference', 'targetType': 'binary', 'environmentID': '5fda0b23d33aebb30a492f74', 'majorVersion': True, 'targetName': 'is_bad', 'positiveClassLabel': 1, 'negativeClassLabel': 0, 'predictionThreshold': 0.17, 'language': 'Python', 'description': None, 'id': None, 'versionId': None, 'trainingData': '/Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_Lending_Club_Loans_utf-8.csv', 'modelVersionID': None, 'testData': '/Users/timothy.whittaker/Desktop/git/ford-mlops/data/10K_LC_short.csv', 'environmentVersionID': '5fda0b242d986b76e2d33bf8'}
__main__ - INFO - 2020-12-16 08:42:21,864 - create new inference 

# Test Model in DataRobot

In [85]:
!python test.py --model-dir ./model --logging-level INFO --max-wait 600

Namespace(logging_level='INFO', max_wait=600, model_dir='./model')
__main__ - INFO - 2020-12-16 09:15:25,239 - Namespace(logging_level='INFO', max_wait=600, model_dir='./model')
__main__ - INFO - 2020-12-16 09:15:25,239 - loading model config
__main__ - INFO - 2020-12-16 09:15:25,246 - starting custom model test
__main__ - INFO - 2020-12-16 09:19:54,738 - test complete
__main__ - INFO - 2020-12-16 09:19:54,738 - warning 😱
__main__ - ERROR - 2020-12-16 09:19:54,738 - Test: error_check
__main__ - ERROR - 2020-12-16 09:19:54,738 - Status: succeeded
__main__ - ERROR - 2020-12-16 09:19:54,738 - Message: 
__main__ - ERROR - 2020-12-16 09:19:54,738 - Test: null_value_imputation
__main__ - ERROR - 2020-12-16 09:19:54,738 - Status: warning
__main__ - ERROR - 2020-12-16 09:19:54,738 - Message: Model cannot impute null values for the following columns:
int_rate, zip_code, addr_state
__main__ - ERROR - 2020-12-16 09:19:54,738 - Test: long_running_service
__main__ - ERROR - 2020-12-16 09:19:54,738 